## 앙상블 - 랜덤포레스트

#### 전체 샘플에서 랜덤 샘플을 선택하는 방식으로, 그레디언트 디센트의 샘플링과 동일함

#### 하지만, 선택된 샘플이 다시 세트로 돌아오며 이 방법을 부트 스트랩 샘플링이라고 함

#### 깊이가 얕고 트리를 많이 생성하는게 목적이다보니, 루트를 씌운 갯수만큼의 특성만 사용하여 최선의 분할을 선택해서 가지치기를 함

#### 그 다음, 각 트리의 클래스별 확률을 평균내어 가장 높은 확률을 가진 클래스를 예측값으로 사용

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [5]:
input = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(input, target, random_state=42, test_size=0.2)

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

In [8]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [9]:
## 훈련값 확인
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [12]:
# OOB 샘플 테스트
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리

In [13]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [14]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레디언트 부스팅

#### 그레디언트 부스팅은 경사하강 알고리즘에서 학습법을 트리 모델을 사용하여 손실을 줄이는 알고리즘

#### 깊이가 3인 트리를 100개 만드는 디폴트값을 가지고 있음

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

In [16]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


#### 그레디언트 부스팅은 분류기 중에서 손실함수의 최적화된 SGD와 분류 성능이 강력한 트리를 앙상블 하여 오버피팅에도 강하고 성능도 매우 강력함

In [18]:
# 트리 개수를 500개로 늘리고, 학습 반영률을 0.2로 조정
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [19]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15853457 0.68010884 0.1613566 ]


## 히스토그램 기반 부스팅

In [20]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [21]:
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


#### 퍼뮤테이션 임포턴스는 첫번째 특성을 무작위로 섞고, 그 외의 특성을 무작위로 섞은 뒤 기존의 데이터셋의 점수와 얼만큼의 차이가 나는지 확인하는 방법론

#### 차이가 클 수록 해당 특성이 모델이 미치는 영향이 크다는 것을 증명함

In [23]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


## XGBoost

In [25]:
from xgboost import XGBClassifier

In [28]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [30]:
from lightgbm import LGBMClassifier

In [31]:
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
